# Y~G ACGT Probabilities 1d
> 

In [ ]:
import numpy as np
import pandas as pd

from EnvDL.core import ensure_dir_path_exists 
from EnvDL.dlfn import g2fc_datawrapper, BigDataset, plDNN_general
from EnvDL.dlfn import ResNet1d, BasicBlock1d
from EnvDL.dlfn import LSUV_

import torch
import torch.nn.functional as F # F.mse_loss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger

In [ ]:
# Run settings: 
max_epoch = 20
batch_size = 24

In [ ]:
use_gpu_num = 0

device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

Using cuda device


In [ ]:
cache_path = '../nbs_artifacts/02.22_g2fc_G_ACGT_conv1d/'
save_prefix = [e for e in cache_path.split('/') if e != ''][-1]
ensure_dir_path_exists(dir_path = cache_path)

## Load data

In [ ]:
X = g2fc_datawrapper()
X.set_split()
X.load_all(name_list = ['obs_geno_lookup', 'YMat', 'ACGT',], store=True) 

X.calc_cs('YMat', version = 'np', filter = 'val:train')

Loading and storing default `phno`.


In [ ]:
training_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = False,
    lookup_obs  = X.get('val:train',       ops_string='   asarray from_numpy      '),
    lookup_geno = X.get('obs_geno_lookup', ops_string='   asarray from_numpy      '),
    y =           X.get('YMat',            ops_string='cs asarray from_numpy float')[:, None],
    G =           X.get('ACGT',            ops_string='           from_numpy float'),
    G_type = 'raw',
    send_batch_to_gpu = 'cuda:0'
    ),
    batch_size = batch_size,
    shuffle = True
)


validation_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = False,
    lookup_obs =  X.get('val:test',        ops_string='   asarray from_numpy      '),
    lookup_geno = X.get('obs_geno_lookup', ops_string='   asarray from_numpy      '),
    y =           X.get('YMat',            ops_string='cs asarray from_numpy float')[:, None],
    G =           X.get('ACGT',            ops_string='           from_numpy float'),
    G_type = 'raw',
    send_batch_to_gpu = 'cuda:0'
    ),
    batch_size = batch_size,
    shuffle = False
)

## Test Models

In [ ]:
def Conv1D_Max_block(in_channels, out_channels, kernel_size, stride):
    block = nn.Sequential(
        nn.Conv1d(
            in_channels= in_channels, # second channel
            out_channels= out_channels,
            kernel_size= kernel_size,
            stride= stride
        ), 
        nn.MaxPool1d((kernel_size,), stride=stride)
    )
    return(block)


In [ ]:
from EnvDL.dlfn import Linear_block

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()    

#         def Linear_block(in_size, out_size, drop_pr):
#             block = nn.Sequential(
#                 nn.Linear(in_size, out_size),
#                 nn.ReLU(),
#                 nn.Dropout(drop_pr)
#             )
#             return(block)         
        
        
#         def Conv1D_Max_block(in_channels, out_channels, kernel_size, stride):
#             block = nn.Sequential(
#                 nn.Conv1d(
#                     in_channels= in_channels, # second channel
#                     out_channels= out_channels,
#                     kernel_size= kernel_size,
#                     stride= stride
#                 ), 
#                 nn.MaxPool1d((kernel_size,), stride=stride)
#             )
#             return(block)
        
        self.x_network = nn.Sequential(
            Conv1D_Max_block(4, 4, 3, 2),
            Conv1D_Max_block(4, 4, 3, 2),
            Conv1D_Max_block(4, 4, 3, 2),
            Conv1D_Max_block(4, 4, 3, 2),
            Conv1D_Max_block(4, 4, 3, 2),
            Conv1D_Max_block(4, 4, 3, 2)
        )
        
    
        self.x_pred = nn.Sequential(
                nn.Flatten(),           
                Linear_block(in_size = (4*29), out_size = 32, drop_pr = 0.3),
                nn.Linear(32, 1)
            )

    def forward(self, x):
        out = self.x_network(x)
        pred = self.x_pred(out)
        return pred#, out

#     def forward(self, x):
#         x_out = self.x_network(x)
#         return x_out

model = NeuralNetwork().to(device)

# model(next(iter(training_dataloader))[1][0:5]).shape

In [ ]:
LSUV_(model, next(iter(training_dataloader))[1])

Applying orthogonal init (zero init if dim < 2) to params in 8 module(s).
Applying LSUV to 8 module(s) (up to 10 iters per module):
Module  0 after  2 itr(s) | Mean: -0.485 | Std: 1.000 | <class 'torch.nn.modules.conv.Conv1d'>
Module  1 after  2 itr(s) | Mean: -0.047 | Std: 1.000 | <class 'torch.nn.modules.conv.Conv1d'>
Module  2 after  2 itr(s) | Mean: -0.227 | Std: 1.000 | <class 'torch.nn.modules.conv.Conv1d'>
Module  3 after  2 itr(s) | Mean: -0.686 | Std: 1.000 | <class 'torch.nn.modules.conv.Conv1d'>
Module  4 after  1 itr(s) | Mean:  0.067 | Std: 0.997 | <class 'torch.nn.modules.conv.Conv1d'>
Module  5 after  2 itr(s) | Mean: -0.214 | Std: 1.000 | <class 'torch.nn.modules.conv.Conv1d'>
Module  6 after  1 itr(s) | Mean: -0.070 | Std: 0.955 | <class 'torch.nn.modules.linear.Linear'>
Module  7 after  6 itr(s) | Mean:  2.065 | Std: 1.092 | <class 'torch.nn.modules.linear.Linear'>


In [ ]:
DNNG = plDNN_general(model)     
optimizer = DNNG.configure_optimizers()

logger = TensorBoardLogger("tb_ACGT_logs", name=save_prefix)
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

In [ ]:
save_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())

pt_path = cache_path+''.join([save_prefix,'__'+save_time,'.pt'])

torch.save(DNNG.mod, pt_path)

In [ ]:
# torch.save(DNNG.mod, cache_path+'g-acgt-conv1'+'.pt')